In [1]:

import time
import threading
from threading import Thread
import random
import sys

## Deadlocks

Deadlocks are yet another problematic condition that might arise as the result of poorly synchronized code.

## A simple example

Let's start by analyzing a simple example: movement between two "bank accounts":

In [2]:
ACCOUNTS = {
    'a1': 1000,
    'a2': 1000
}
ITERATIONS = {
    'a1': 0,
    'a2': 0,
}

In [3]:
def move_funds(_from, _to, expected):
    name = threading.current_thread().name

    while True:
        amount = random.randint(0, 100)
        ACCOUNTS[_from] -= amount
        ACCOUNTS[_to] += amount
        total = sum(ACCOUNTS.values())
        if total != expected:
            print(f"{name} found an inconsistent balance: ${total}")
            break
        ITERATIONS[_from] += 1

In [4]:
t1 = threading.Thread(target=move_funds, name='Thread-1', args=('a1', 'a2', 2000))
t2 = threading.Thread(target=move_funds, name='Thread-2', args=('a2', 'a1', 2000))

In [5]:
t1.start()
t2.start()

Thread-2 found an inconsistent balance: $1930


## Fixing it with Locks

We've already learned about Locks, so we can use those to try synchronizing the access to the accounts. We'll create 2 locks, on for each account:

In [6]:
ACCOUNTS = {
    'a1': 1000,
    'a2': 1000
}

In [7]:
lock_1 = threading.Lock()
lock_2 = threading.Lock()

In [8]:
def move_funds(_from, _to, lock_from, lock_to, expected):
    name = threading.current_thread().name
    iterations = 0
    while True:
        amount = random.randint(0, 100)
        with lock_from:
            with lock_to:
                ACCOUNTS[_from] -= amount
                ACCOUNTS[_to] += amount
                total = sum(ACCOUNTS.values())
                if total != expected:
                    print(f"{name} found an inconsistent balance: ${total}")
                    break
        iterations += 1
        if iterations > 1_000_000:
            print(f'{name} reached iteration limit. Stopping...')
            return

In [9]:
t1 = threading.Thread(target=move_funds, name='Thread-1', args=('a1', 'a2', lock_1, lock_2, 2000))
t2 = threading.Thread(target=move_funds, name='Thread-2', args=('a2', 'a1', lock_1, lock_2, 2000))

In [10]:
t1.start()
t2.start()

Thread-1 found an inconsistent balance: $1970
Thread-1 found an inconsistent balance: $1904


In [11]:
[t.join() for t in (t1, t2)];

Thread-2 reached iteration limit. Stopping...


In [12]:
sum(ACCOUNTS.values())

2000

In [13]:
lock_1.locked(), lock_2.locked()

(False, False)

Worked! Now the access to the accounts is protected by the locks. But there's a very dangerous potential situation hidden in our code. If we make just 1 tiny change, altering the order of the locks that are passed to our threads, we'll find ourselves deadlocked:

In [14]:
ACCOUNTS = {
    'a1': 1000,
    'a2': 1000
}

In [41]:
# t1 = threading.Thread(target=move_funds, name='Thread-1', args=('a1', 'a2', lock_1, lock_2, 2000))
# t2 = threading.Thread(target=move_funds, name='Thread-2', args=('a2', 'a1', lock_2, lock_1, 2000))
# t1.start()
# t2.start()

# [t.join() for t in (t1, t2)];

## What are Deadlocks and when do they happen?

a deadlock is a state in which each member of a group is waiting for another member, including itself, to take action, such as sending a message or more commonly releasing a lock

![title](deadlock.gif)

In our code, the issue is the ordering in which the locks are acquired. This is a VERY difficult thing to spot.

![title](deadlocks.png)

## How to prevent Deadlock

 It's very hard to prevent Deadlocks. One simple technique is to always use timeouts when trying to acquire locks. If you're trying to acquire N shared locks, if you can't acquire all N, you can release them all and start over. Let's see an example:

In [15]:
ACCOUNTS = {
    'a1': 1000,
    'a2': 1000
}

LOCK_TIMEOUT = .001

lock_1 = threading.Lock()
lock_2 = threading.Lock()

In [16]:
def move_funds(_from, _to, lock_from, lock_to, expected):
    name = threading.current_thread().name
    iterations = 0
    while True:
        amount = random.randint(0, 100)
        locked = False
        while not locked:
            res1 = lock_from.acquire(timeout=LOCK_TIMEOUT)
            res2 = lock_to.acquire(timeout=LOCK_TIMEOUT)
            if all([res1, res2]):
                # Success! We acquired both locks
                locked = True
            else:
                # Release locks "partially" acquired
                if res1:
                    lock_from.release()
                if res2:
                    lock_to.release()
        # locked is True, we're safe
        ACCOUNTS[_from] -= amount
        ACCOUNTS[_to] += amount
        total = sum(ACCOUNTS.values())
        lock_from.release()
        lock_to.release()
        if total != expected:
            print(f"{name} found an inconsistent balance: ${total}")
            break
        iterations += 1
        if iterations > 100_000:
            print(f'{name} reached iteration limit. Stopping...')
            return

In [17]:
t1 = threading.Thread(target=move_funds, name='Thread-1', args=('a1', 'a2', lock_1, lock_2, 2000))
t2 = threading.Thread(target=move_funds, name='Thread-2', args=('a2', 'a1', lock_2, lock_1, 2000))

In [18]:
t1.start()
t2.start()

In [19]:
[t.join() for t in (t1, t2)];

Thread-1 reached iteration limit. Stopping...
Thread-2 reached iteration limit. Stopping...


In [20]:
sum(ACCOUNTS.values())

2000

In [21]:
lock_1.locked(), lock_2.locked()

(False, False)

In [ ]:
As you can see, we've just prevented the deadlock. The key code piece that prevents the deadlock is the following:

In [ ]:
# locked = False
# while not locked:
#     res1 = lock_from.acquire(timeout=LOCK_TIMEOUT)
#     res2 = lock_to.acquire(timeout=LOCK_TIMEOUT)
#     if all([res1, res2]):
#         # Success! We acquired both locks
#         locked = True
#     else:
#         # Release locks "partially" acquired
#         if res1:
#             lock_from.release()
#         if res2:
#             lock_to.release()